In [ ]:
import os
import pandas as pd
import warnings
import openpyxl
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

def get_files_names(folder_path):
    files = os.listdir(folder_path)
    files_path = []
    for file in files:
        file = os.path.join(folder_path, file)
        files_path.append(file)
    return files_path

dir_actual = os.getcwd()
folder_path = os.path.join(dir_actual, 'Backup')
files = get_files_names(folder_path)


In [ ]:
def get_dataset_NLP_V2(excel_path):
    excels_errors = {}
    try:
        # Cargar el libro y verificar la hoja
        libro = openpyxl.load_workbook(excel_path, data_only=True)
        if "004" not in libro.sheetnames:
            excels_errors[excel_path] = 'La hoja "004" no existe en el archivo Excel.'
            return pd.DataFrame(), excels_errors
        
        hoja = libro["004"]

        # Buscar la fila que contiene "descrip"
        start_row = None
        for fila in hoja.iter_rows(min_row=1, max_row=15, min_col=1, max_col=5):
            for celda in fila:
                if celda.value and "descrip" in str(celda.value).lower():
                    nombre_partida = celda.value
                    start_row = celda.row
                    break
            if start_row:
                break

        if not start_row:
            excels_errors[excel_path] = 'No se encontró ninguna celda con "descrip".'
            return pd.DataFrame(), excels_errors

        # Leer el DataFrame a partir de la fila encontrada
        df = pd.read_excel(excel_path, sheet_name="004", skiprows=start_row-1)

        if "Cod." not in df.columns:
            excels_errors[excel_path] = 'No se encontró la columna "Cod." en la hoja "004".'
            return pd.DataFrame(), excels_errors

        df = df.rename(columns={nombre_partida: 'Descripcion', "Cod.": "Codigo"})
        df = df[[nombre_partida, "Codigo"]].dropna().reset_index(drop=True)
        excels_errors[excel_path] = 'OK'
        df['archivo'] = excel_path
        return df, excels_errors

    except Exception as e:
        excels_errors[excel_path] = f'Error inesperado: {str(e)}'
        return pd.DataFrame(), excels_errors


In [ ]:
dfs = []
errores = {}

for file in files:
    df, errores_temp = get_dataset_NLP_V2(file)
    dfs.append(df)
    errores.update(errores_temp)

df_final = pd.concat(dfs, ignore_index=True)
registro = pd.DataFrame(errores.items(), columns=['archivo', 'error'])


In [ ]:
def export_excels_tabs(df, registro, path):
    with pd.ExcelWriter(path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='NLP', index=False)
        registro.to_excel(writer, sheet_name='Registro', index=False)

export_excels_tabs(df_final, registro, 'NLP_V2.xlsx')